#### Loading preprocessed data

In [1]:
import pickle
file_name = '/content/preprocessed_food_reviews.pkl'
with open(file_name, 'rb') as file:
    df_food_review = pickle.load(file)

In [2]:
df_food_review.head(3)

,Text,Score,clean_text
0,This product was horrible. The very first can...,0,this product be horrible the very first can ...
1,My cat usually loves Fancy Feast wet food and ...,0,my cat usually love fancy feast wet food and f...
2,"I really love this Pu'erh tea. It is rich, dar...",1,I really love this puerh tea it be rich dark a...


In [3]:
x = df_food_review.drop(columns='Score', axis=1)
y = df_food_review['Score']
# split data to train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [4]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((13139, 2), (3285, 2), (13139,), (3285,))

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
# load the model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
train_embeddings = model.encode(x_train['clean_text'].to_list(), show_progress_bar=True)
test_embeddings = model.encode(x_test['clean_text'].to_list(), show_progress_bar=True)

Batches:   0%|          | 0/411 [00:00<?, ?it/s]

Batches:   0%|          | 0/103 [00:00<?, ?it/s]

In [8]:
from sklearn.metrics import classification_report
def evaluate_performance(y_true, y_pred):
  performance = classification_report(y_true, y_pred, target_names=['Negative Review', 'Positive Review'])
  print(performance)

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
logistic_regression_clf = LogisticRegression(random_state=42)
logistic_regression_clf.fit(train_embeddings, y_train)

LogisticRegression(random_state=42)

In [11]:
y_train_predict = logistic_regression_clf.predict(train_embeddings)

In [12]:
evaluate_performance(y_train, y_train_predict)

                 precision    recall  f1-score   support

Negative Review       0.88      0.88      0.88      6596
Positive Review       0.88      0.88      0.88      6543

       accuracy                           0.88     13139
      macro avg       0.88      0.88      0.88     13139
   weighted avg       0.88      0.88      0.88     13139



In [13]:
y_pred = logistic_regression_clf.predict(test_embeddings)

In [14]:
evaluate_performance(y_test, y_pred)

                 precision    recall  f1-score   support

Negative Review       0.88      0.87      0.87      1616
Positive Review       0.88      0.88      0.88      1669

       accuracy                           0.88      3285
      macro avg       0.88      0.88      0.88      3285
   weighted avg       0.88      0.88      0.88      3285



In [15]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(train_embeddings, y_train)

GaussianNB()

In [16]:
y_train_pred_nb = nb.predict(train_embeddings)

In [17]:
evaluate_performance(y_train, y_train_pred_nb)

                 precision    recall  f1-score   support

Negative Review       0.82      0.78      0.80      6596
Positive Review       0.79      0.82      0.80      6543

       accuracy                           0.80     13139
      macro avg       0.80      0.80      0.80     13139
   weighted avg       0.80      0.80      0.80     13139



In [18]:
y_pred_nb = nb.predict(test_embeddings)
evaluate_performance(y_test, y_pred_nb)

                 precision    recall  f1-score   support

Negative Review       0.81      0.77      0.79      1616
Positive Review       0.79      0.82      0.80      1669

       accuracy                           0.80      3285
      macro avg       0.80      0.80      0.80      3285
   weighted avg       0.80      0.80      0.80      3285



# SVM linear

In [19]:
from sklearn.svm import LinearSVC

In [20]:
linear_svc = LinearSVC()
linear_svc.fit(train_embeddings, y_train)

LinearSVC()

In [21]:
y_train_pred = linear_svc.predict(train_embeddings)
evaluate_performance(y_train, y_train_pred)

                 precision    recall  f1-score   support

Negative Review       0.90      0.91      0.90      6596
Positive Review       0.91      0.90      0.90      6543

       accuracy                           0.90     13139
      macro avg       0.90      0.90      0.90     13139
   weighted avg       0.90      0.90      0.90     13139



In [22]:
y_pred_test = linear_svc.predict(test_embeddings)
evaluate_performance(y_test, y_pred_test)

                 precision    recall  f1-score   support

Negative Review       0.89      0.89      0.89      1616
Positive Review       0.89      0.90      0.89      1669

       accuracy                           0.89      3285
      macro avg       0.89      0.89      0.89      3285
   weighted avg       0.89      0.89      0.89      3285



# K-fold Cross Validation

In [23]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

# Logistic Regression cross validation

In [25]:
# Define the number of splits for k-fold
k = 5  # 5-fold cross-validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)
model1 = logistic_regression_clf
# Perform k-fold cross-validation
# Accuracy is the default scoring metric for classification
scores = cross_val_score(model1, test_embeddings, y_test, cv=kf, scoring='accuracy')

# Print the results
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean():.4f}")
print(f"Standard deviation: {scores.std():.4f}")

Cross-validation scores: [0.86757991 0.85692542 0.85844749 0.83866058 0.87062405]
Mean accuracy: 0.8584
Standard deviation: 0.0112


# Linear SVC cross validation

In [26]:
# Define the number of splits for k-fold
k = 5  # 5-fold cross-validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)
model2 = linear_svc

# Perform k-fold cross-validation
# Accuracy is the default scoring metric for classification
scores = cross_val_score(model2, test_embeddings, y_test, cv=kf, scoring='accuracy')

# Print the results
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean():.4f}")
print(f"Standard deviation: {scores.std():.4f}")

Cross-validation scores: [0.88432268 0.86605784 0.87366819 0.847793   0.88280061]
Mean accuracy: 0.8709
Standard deviation: 0.0133


### Based on the findings it is evident that **Support Vector Machine (Linear SVC)** model has good prediction performance with **sentence-transformers/all-mpnet-base-v2** embedding model

In [27]:
# saving trained model
import pickle
file_name = 'Linear_svc_model.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(linear_svc, file)
print(f"trained model saved as : {file_name}")

trained model saved as : Linear_svc_model.pkl
